In [1]:
#第一步调包
#以下调的包功能有重复，但本次工程以总结思路为主，所以尽量在调包时清晰全面的理清特征工程思路，调包主要是因为专业的事情尽量交给专业的人去做，如果
#有现成的轮子就不用自己造了，但是基本原理需要理解

#数据处理包
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#处理下负号显示问题
plt.rcParams['axes.unicode_minus'] = False

In [2]:
#按照特征工程思路调机器学习包
#首先调特征工程包
import sklearn
#1.处理异常值

#  前言
#  对于非缺失值，无可用模块，需要自行处理或者利用算法本身性质处理
#  对于缺失值，使用以下模块或者功能
#  常用填充的缺失取值方式
#  所有缺失值填充相同内容: mean，mode，median，unknown
#  所有缺失值填充不同内容:热卡填充(选择主观相似样本)，K-means：选择K个临近的样本属性值的均值（临近:欧式距离、hamming距离。。），线性回归，
#  EM最大化(不太理解，但好像也不是很好用)，多重插补（资料少，不是很理解）

#  调包
#  1) pandas.DataFrame的fillna功能,用来自定义缺失值
pd.DataFrame([]).fillna(0)
#  2）pandas.DataFrame的dropna功能，用来删除缺失值
pd.DataFrame([]).dropna()
#  3）使用sklearn插值模块impute中的SimpleImputer，作用和Imputer类似
from sklearn.impute import SimpleImputer
#  4) 使用sklearn插值模块impute中的IterativeImputer，这种方式类似多重插补，或者EM最大化。使用迭代的方式利用其它feature值产生各feature的缺失值
#  目前还在测试阶段，所以需要使能测试
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
#  5) 使用sklearn插值模块impute中的KNNImputer,这是一种K-means填充法，使用的距离为欧式距离
from sklearn.impute import KNNImputer
#  6) 使用sklearn插值模块impute中的MissingIndicator,用来寻找缺失值位置
from sklearn.impute import MissingIndicator


In [3]:
#2.类别型特征处理

#  前言
#  类别型特征处理前先看文本型特征能否归纳为普通类别型特征。
#  长文本型特征处理方式: python字符串分割方法，正则表达式匹配，词袋（所有词建立词袋，每个样本的文本对应一个词频array），N-gram（相当于
#  N个连续词组的词袋），TF-IDF（TF:相当于该段文本中该词出现频率越高说明这个词越重要，IDF:相当于该词在所有文本中出现频率的倒数越高说明
#  这个词信息量越大，TF*IDF为这个词在该段文本中的重要性）
#
#  普通类别型特征处理方式:labelencoder,OneHot,直方图/计数编码（使用每个类别值对标记的影响进行编码，不是很好用但是是一些其他编码方式的
#  基础思想），WOE(ln(每个类别值负分类比例/每个类别值正分类比例))，IV(每个类别值负分类比例/每个类别值正分类比例-WOE，为减小样本比较少的属
#  性值的误差)，目标编码（某属性值下样本很多，则偏向相信这个属性值对应的正标记值概率，反之则更相信全局正标记值概率），均值编码（和目标编码
#  类似，处理回归问题，把正值概率转化为标记的均值）

#  调包
#  1) python字符串分割方法
#  略
#  2）正则表达式匹配，导入re模块
import re
#  3) 从sklearn特征抽取模块feature_extraction模块中导入DictVectorizer,字典
from sklearn.feature_extraction import DictVectorizer
#  4) 从sklearn特征抽取模块feature_extraction模块中的text类中导入DictVectorizer,词袋
from sklearn.feature_extraction.text import CountVectorizer
#  5) 从sklearn特征抽取模块feature_extraction模块中的text类中导入TfidfVectorizer,tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer


#  6) 从sklearn预处理模块preprocessing模块中导入OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
#  7) 从sklearn预处理模块preprocessing模块中导入LabelEncoder
from sklearn.preprocessing import LabelEncoder
#  9）从类别编码模块category_encoder中导入目标编码类TargetEncoder
from category_encoders import TargetEncoder
#  10) 从类别编码模块category_encoder中导入目标编码类MEstimateEncoder应该就是均值编码了吧
from category_encoders import MEstimateEncoder

In [4]:
#3.特征筛选和降维
#  前言
#  该步骤目的是尽量减少特征列的数目并且保证训练样本的信息不丢失或者丢失的少，这样尽量减小了过拟合，我把特征筛选和降维放到一起是因为我认为这两
#  个步骤是在完成同一件事情，只不过采取的手段不太相同，特征筛选主要是选择可以直接删除的特征列，而降维则是某种合并多种特征的方式。
#
#  特征筛选有几种风格处理方式：首先是应该基于专业分析和经验的筛选以及组合特征；然后是根据某些指标选取一定数量的特征用来保留，或者说选择一些特
#  征来删除（filter）；三是使用算法训练若干迭代每个迭代也训练若干小轮，时间复杂度O（n*n*O(算法)）每个迭代选出一个增加后拟合效果最好的特征加入
#  特征池，每个迭代的每个小轮使用不在特征池中的一个特征暂时加入特征池训练后评分（wrapper）；四是在算法本身就有进行特征筛选的设计，比如说正则化
#  项，或者剪枝等（Embeded）
#
#  降维有两个方式PCA和LDA，在特征之提取主成分，主成分可以理解为方差最大的那个特征，亦或是和其他特征向量线性相关的某个方差最大的特征向量

#  调包
#  1) 基于专业和经验筛选特征
#  略
#  2）从sklearn的feature_selection模块中导入SelectKBest，属于filter，过滤标准为自定义
from sklearn.feature_selection import SelectKBest
#  3）从sklearn的feature_selection模块中导入VarianceThreshold，属于filter，过滤标准为选择方差尽量大的特征
from sklearn.feature_selection import VarianceThreshold
#  4）从scipy模块的统计库stats中导入相关系数计算，可以用来使用相关系数作为指标筛选特征
from scipy.stats import pearsonr
#  5）从sklearn的feature_selection模块中导入chi2，属于filter，用卡方检验，卡方检验核心思想是，如果卡方值越小说明当属性变化时标记变化的越明显
from sklearn.feature_selection import chi2
#  6）导入一个互信息函数MINE作为评价标准，因为没有安装该模块也暂时不打算使用
#  from minepy import MINE
#  7）从sklearn的特征选择模块中导入RFE,核心为wrapper思想，迭代通过学习生成最好的特征，本次作业打算使用lgb.LGBMClassifier()算法，基算法和决策
#  树模型相近，所以这里也导入DecisionTreeClassifier来进行检验
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
#  8) Embedded
#  略
#  9） 从sklearn降维库decomposition导入PCA算法
from sklearn.decomposition import PCA
#  10） 从sklearn降维库decomposition导入LDA算法
from sklearn.decomposition import LatentDirichletAllocation


In [5]:
#4.标准化归一化
# 前言
# 过于简单，归一化令所有属性值分布到0,1之间，标准化令所有属性值归到零附近，亦或者说尽量让所有属性值归成正太分布

#  调包
# 1）调归一化包
from sklearn.preprocessing import MinMaxScaler
# 2）调标准化包
from sklearn.preprocessing import StandardScaler

In [35]:
#然后调机器学习算法包，本次项目拟采用LGBMClassifier算法
import lightgbm as lgb
from lightgbm import LGBMClassifier 
#交叉验证
from sklearn.model_selection import GridSearchCV
#分样本集
from sklearn.model_selection import train_test_split

In [7]:
#保存模型工具
import pickle 

好了基本上特征工程思路理清了，下面就开始继续理清工程思路  
1. 首先肯定是要提取rawdata，为了结构清晰处理方便，我们需要将test data 和train data都在开始提取，并且将traindata的目标值数据和特征值数据分离。这样后面可以在进行特征工程时处理完train data 直接处理testdata  
2. 特征工程的处理我认为该按照如下顺序更合适，这样每一步的执行都有利于将下一步的操作简化:  
    1)根据经验和专业删除特征  
    2)文本型特征处理  
    3)类别型特征处理    
    4)补充缺失值  
    5)标准化  
    6)降维  
    7)筛选特征  
3. 设定大致的模型参数范围
4. 进行交叉验证
5. 使用最佳参数进行训练
6. 保存模型备份
7. 使用训练好的模型进行拟合
8. 将拟合结果整理保存

## 开始第1步，提取rawdata ##

In [8]:
# read csv
raw_train_data = pd.read_csv(r'C:\Users\ZOE ZHAO\Desktop\Course\lesson8 特征工程\train.csv')
raw_test_data = pd.read_csv(r'C:\Users\ZOE ZHAO\Desktop\Course\lesson8 特征工程\test.csv')

#分离训练数据中的特征和标签
train_target = raw_train_data.loc[:,"Survived"]
raw_train_data.drop(["Survived"],inplace=True,axis = 1)

#分离训练数据和测试数据的ID
train_ID = raw_train_data.loc[:,"PassengerId"]
raw_train_data.drop(["PassengerId"],inplace=True,axis = 1)
test_ID = raw_test_data.loc[:,"PassengerId"]
raw_test_data.drop(["PassengerId"],inplace=True,axis = 1)

In [9]:
raw_train_data

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
886,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [10]:
raw_test_data.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## 第二步特征工程 ##

In [11]:
# 1)根据经验和专业删除特征
# 以我的经验和学识来说，实在无法理解Ticket号码背后蕴藏的规律，故舍弃
# 当然后续还可以挖掘一下Ticket编码长度、前缀等信息和生存率之间的关系，但本次项目为了简化先不予考虑
# 数据集变量命名规则:train/test_data_xy, x是大步骤数，y是小步骤数
train_data_21 = raw_train_data.drop(["Ticket"],axis = 1)
test_data_21 = raw_test_data.drop(["Ticket"],axis = 1)

In [12]:
# 2)文本特征处理
# 属于文本特征的就是Name，和Cabin该列唯一可以提取的是名字中间的称谓，可以看到称谓和Sex有很强的相关性，但是我们先不去管它，只管把称谓提取出来，后
# 面做特征提取和降维的时候自然会消除这种影响
# 这里偷个懒，直接copy下大神的代码
# 大神代码里加入了surname列，我认为对于该数据集来说surname单一类别的样本太少，还是删除比较好

# -------------------------------------以下开始大神代码---------------------------------------------------
def split_name_str(s, title_map):
    """
    Extract title from name, replace with value in title dictionary and both title 
    and surname.
    """

    # Remove '.' from name string
    s = s.replace('.', '')
    # Split on spaces
    s = s.split(' ')
    # get surname
    surname = s[0]

    # Get title - loop over title_map, if s matches a key, take the
    # corresponding value as the title
    title = [t for k, t in title_map.items() if str(k) in s]

    # If no matching keys in title dict, use 'Other'.
    if len(title) == 0:
        title = 'Other'
    else:
        # Title is a list, so extract contents
        title = title[0]

    # Return surname (stripping remaining ',') and title as string
    return surname.strip(','), title


title_map_ = {"Capt": "Officer", "Col": "Officer", "Major": "Officer", 
              "Jonkheer": "Sir", "Don": "Sir", "Sir": "Sir", "Dr": "Dr", 
              "Rev": "Rev", "theCountess": "Lady", "Dona": "Lady", 
              "Mme": "Mrs", "Mlle": "Miss", "Ms": "Mrs", "Mr": "Mr", 
              "Mrs": "Mrs", "Miss": "Miss", "Master": "Master",
              "Lady": "Lady"}

# Apply functions to df and concatenate new columns as before
train_data_22 = train_data_21['Name'].apply(split_name_str, args=[title_map_])
train_data_22 = train_data_22.apply(pd.Series)
train_data_22.columns = ['Surname', 'Title']
train_data_22 = pd.concat([train_data_21, train_data_22], axis=1)
train_data_22.drop(['Name','Surname'], axis=1,inplace=True)

test_data_22 = test_data_21['Name'].apply(split_name_str, args=[title_map_])
test_data_22 = test_data_22.apply(pd.Series)
test_data_22.columns = ['Surname', 'Title']
test_data_22 = pd.concat([test_data_21, test_data_22], axis=1)
test_data_22.drop(['Name','Surname'], axis=1,inplace=True)



#以下处理Cabin列
def split_cabin(s):
    match = re.match(r"([a-z]+)([0-9]+)",s,re.I)
    letter = ''
    number = -1.0
    if match is not None:
        letter = match.group(1)
        number = match.group(2)
        return letter, number
    
    #改进大神算法，处理只有字母没有数字的情况
    match = re.match(r"([a-z]+)",s,re.I)
    if match is not None:
        letter = match.group(1)
    return letter, number

#s:NaN or like "C1 C2"format
#这里我改进了一下大神的算法，大神算法没法正确处理类似“F G12”形式的数据
def process_cabin(s):
    if not isinstance(s, str):
        return '','',-1.0
    else:
        
        # 处理“F G12”形式的数据，目标：删除字母间的空格，但保留字母和数字间的空格
        #将字符串转化成列表方便处理
        temp_str_list = list(s)
        #设定一个list用来记录需要删除空格的index
        temp_del_list = list()
        #搜索符合条件的空格
        for idx_i in range(0,len(temp_str_list)):
            if temp_str_list[idx_i]==' ' and len(temp_str_list)>0 and idx_i<len(temp_str_list)-1 and idx_i>0:
                if temp_str_list[idx_i-1].isalpha() and temp_str_list[idx_i+1].isalpha():
                    #记录需要删除的index
                    temp_del_list+=[idx_i]
        #删除相应空格
        for idx_i in temp_del_list:
            del temp_str_list[idx_i]
        #重新将list转化为str
        s = ''.join(temp_str_list)
        # 后面是正常的大神代码
        
        s = s.split()
        num = len(s)
        s = s[0]
    l,n = split_cabin(s)
    return l,n,num

cabin_t = train_data_22.loc[:,'Cabin'].apply(process_cabin)
cabin_t = cabin_t.apply(pd.Series)
cabin_t.columns = ['cabin_letter', 'cabin_number', 'cabins_number_of']

train_data_22 = train_data_22.drop('Cabin',axis=1)
train_data_22 = pd.concat([train_data_22, cabin_t], axis=1)

cabin_t = test_data_22.loc[:,'Cabin'].apply(process_cabin)
cabin_t = cabin_t.apply(pd.Series)
cabin_t.columns = ['cabin_letter', 'cabin_number', 'cabins_number_of']

test_data_22 = test_data_22.drop('Cabin',axis=1)
test_data_22 = pd.concat([test_data_22, cabin_t], axis=1)
# -------------------------------------以上大神代码---------------------------------------------------

#将各行填入正确格式的空缺值而不是类似'',-1之类的数据
cutter = (test_data_22['cabins_number_of']==-1)
test_data_22.loc[cutter,'cabins_number_of'] = np.nan
cutter = (test_data_22['cabin_letter']=='')
test_data_22.loc[cutter,'cabin_letter'] = np.nan
cutter = (test_data_22['cabin_number']=='')
test_data_22.loc[cutter,'cabin_number'] = np.nan

cutter = (train_data_22['cabins_number_of']==-1)
train_data_22.loc[cutter,'cabins_number_of'] = np.nan
cutter = (train_data_22['cabin_letter']=='')
train_data_22.loc[cutter,'cabin_letter'] = np.nan
cutter = (train_data_22['cabin_number']=='')
train_data_22.loc[cutter,'cabin_number'] = np.nan


#这里再进一步对增加的三行进行分析cabin_letter可能和位置有关所以认为比较重要，cabin_number也可能和位置有关，但不知道相关性如何所以暂时都保留
#对于cabins_number_of，先看大概的分布情况
#等于1的特征值
print((test_data_22['cabins_number_of']==1).sum())
#大于1的特征值
print((test_data_22['cabins_number_of']>1).sum())
#大于2的特征值
print((test_data_22['cabins_number_of']>2).sum())
#打印结果
#74
#17
#7
#发现各特征值样本数相差不到一个量级，但是总样本数又太少，所以我们从保留信息的角度我们应该保留这三行的数据同时看看是否需要在后面想办法减小这三
#行数据的权重。

77
14
7


In [13]:
train_data_22

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,cabin_letter,cabin_number,cabins_number_of
0,3,male,22.0,1,0,7.2500,S,Mr,NaN,NaN,NaN
1,1,female,38.0,1,0,71.2833,C,Mrs,C,85,1.0
2,3,female,26.0,0,0,7.9250,S,Miss,NaN,NaN,NaN
3,1,female,35.0,1,0,53.1000,S,Mrs,C,123,1.0
4,3,male,35.0,0,0,8.0500,S,Mr,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
886,2,male,27.0,0,0,13.0000,S,Rev,NaN,NaN,NaN
887,1,female,19.0,0,0,30.0000,S,Miss,B,42,1.0
888,3,female,NaN,1,2,23.4500,S,Miss,NaN,NaN,NaN
889,1,male,26.0,0,0,30.0000,C,Mr,C,148,1.0


In [14]:
test_data_22

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,cabin_letter,cabin_number,cabins_number_of
0,3,male,34.5,0,0,7.8292,Q,Mr,NaN,NaN,NaN
1,3,female,47.0,1,0,7.0000,S,Mrs,NaN,NaN,NaN
2,2,male,62.0,0,0,9.6875,Q,Mr,NaN,NaN,NaN
3,3,male,27.0,0,0,8.6625,S,Mr,NaN,NaN,NaN
4,3,female,22.0,1,1,12.2875,S,Mrs,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
413,3,male,NaN,0,0,8.0500,S,Mr,NaN,NaN,NaN
414,1,female,39.0,0,0,108.9000,C,Lady,C,105,1.0
415,3,male,38.5,0,0,7.2500,S,Mr,NaN,NaN,NaN
416,3,male,NaN,0,0,8.0500,S,Mr,NaN,NaN,NaN


In [15]:
#3)分类特征处理
# 我认为思路是应该先看那些分类特征可以直接用lableEncoder处理，如果能直接编码那其实是非常方便稳定的处理方式，我认为，对于任何二值型分类特征，都
# 可以直接进行lableEncoder，因为可以默认为他们之间存在某种次序关系，当我们不关心次序的升降时，可以将他们直接进行编码，所以Sex项可以进行lableE
# coder，对于title，我认为理论上也是可以进行lableencoder的，但是因为对于称谓的专业知识不足，这里也不进行lableencoder.对于Pclass，显然也是可以
# 进行lableEncoder的，但是鉴于已经以数字形式记录，则不处理

#由于从该项工作开始，我们在处理某项特征时，也不得不考虑该项特征值和目标值以及其他特征值的关系，所以我们有必要暂时先关注一下缺失值的问题，我想
#要设计一种记录方式，能反映出数据缺失的严重程度，这样根据不同严重程度来进行处理。这使我想到可以设计一个字典，字典的key是特征的名字，字典的值
#为某种缺失率，那么接下来应该探讨一个问题，就是我们应该如何看待测试集上缺失的样本?测试集上面缺失的样本需要在现阶段考虑吗?我认为暂时是不需要的
#因为在测试集上缺失特征值的样本只会影响在测试集上预测的精准度，而不会影响模型本身的性能。所以缺失率应该简单的设计成
#（缺失特征值的样本数）/（样本总数）

#定义空字典：
degree_of_missing = dict.fromkeys(train_data_22.columns,0)
for feature_i in train_data_22.columns:
    degree_of_missing[feature_i] = (train_data_22.loc[:,feature_i]).isnull().sum()/len(train_data_22)
#将特征缺失率按照从大到小排序
degree_of_missing = dict(sorted(degree_of_missing.items(), key = lambda kv:(kv[1], kv[0]),reverse=True))
    
#列出所有需要进行lableEncoder的特征
Lable_encoder_list = ["Sex"]

#其次应该尽量避免使用OneHot，因为我觉得OneHot有点笨重，这里我选择使用TargetEncoder对其他类别进行编码
Target_encoder_list = ["Embarked","Title","cabin_letter"]

#Creat new data for this step
train_data_23 = train_data_22.copy()
test_data_23 = test_data_22.copy()

#进行labelencoder
for feature_i in Lable_encoder_list:
    le = LabelEncoder()
    le.fit(train_data_23.loc[:,feature_i])
    train_data_23.loc[:,feature_i] = le.transform(train_data_23.loc[:,feature_i])
    test_data_23.loc[:,feature_i] = le.transform(test_data_23.loc[:,feature_i])
    
#进行Target_encoder
te = TargetEncoder(cols=Target_encoder_list,handle_missing='return_nan', handle_unknown='return_nan')
te.fit(train_data_23,train_target)
train_data_23 = te.transform(train_data_23)
test_data_23 = te.transform(test_data_23)

In [16]:
train_data_23

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,cabin_letter,cabin_number,cabins_number_of
0,3,1,22.0,1,0,7.2500,0.336957,0.156673,NaN,NaN,NaN
1,1,0,38.0,1,0,71.2833,0.553571,0.795276,0.593220,85,1.0
2,3,0,26.0,0,0,7.9250,0.336957,0.701087,NaN,NaN,NaN
3,1,0,35.0,1,0,53.1000,0.336957,0.795276,0.593220,123,1.0
4,3,1,35.0,0,0,8.0500,0.336957,0.156673,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
886,2,1,27.0,0,0,13.0000,0.336957,0.002569,NaN,NaN,NaN
887,1,0,19.0,0,0,30.0000,0.336957,0.701087,0.744681,42,1.0
888,3,0,NaN,1,2,23.4500,0.336957,0.701087,NaN,NaN,NaN
889,1,1,26.0,0,0,30.0000,0.553571,0.156673,0.593220,148,1.0


In [17]:
test_data_23

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,cabin_letter,cabin_number,cabins_number_of
0,3,1,34.5,0,0,7.8292,0.389610,0.156673,NaN,NaN,NaN
1,3,0,47.0,1,0,7.0000,0.336957,0.795276,NaN,NaN,NaN
2,2,1,62.0,0,0,9.6875,0.389610,0.156673,NaN,NaN,NaN
3,3,1,27.0,0,0,8.6625,0.336957,0.156673,NaN,NaN,NaN
4,3,0,22.0,1,1,12.2875,0.336957,0.795276,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
413,3,1,NaN,0,0,8.0500,0.336957,0.156673,NaN,NaN,NaN
414,1,0,39.0,0,0,108.9000,0.553571,0.383838,0.59322,105,1.0
415,3,1,38.5,0,0,7.2500,0.336957,0.156673,NaN,NaN,NaN
416,3,1,NaN,0,0,8.0500,0.336957,0.156673,NaN,NaN,NaN


In [18]:
#4)补充缺失值

#首先将训练集中的缺失元素的特征补充完整，然后顺便补充测试集的数据，之后完整检查一遍测试集是否还有缺失数据，之后再用训练集中的数据来补充。

#将缺失程度分为两个档次，一种是缺失率<10%的，我认为可以直接用均值或者众数填充，具体取决于是分类型特征还是数值型特征。
#另外一种缺失率>10%的用KNNImputer，

#设计判断类别特征还是数值特征的函数
def Judge_Feature_type(feature_for_testing, raw_data, boundary = 0.1):
    '''
        boundary 越小越容易归类为Number特征
        boundary <= 1 则按照值种类数目在样本总数中的比例来判断
        boundary > 则按照种类数目的绝对值来判断
    ''' 
    fft = feature_for_testing
    #只使用非空样本做判断
    data_not_nan = raw_data.loc[raw_data.loc[:,fft].notnull(),:]
    data_len = len(data_not_nan)
    type_num = len(data_not_nan.drop_duplicates(fft))
    #默认返回class类型
    feature_type = 'Class'
    #如果该特征下特征值的种类很多，多于boundary，则将该特征判定为数值型特征，否则是分类特征
    if boundary <= 1:
        boundary *= data_len
    if type_num >= boundary:
        feature_type = 'Number'
    return feature_type

#定义均值填充特征
mean_impute_features_list = [x for x in degree_of_missing.keys() 
                        if (degree_of_missing[x]>0 
                            and degree_of_missing[x]<=0.1 
                            and Judge_Feature_type(x,train_data_23)=='Number')]
#定义众数填充特征
mode_impute_features_list = [x for x in degree_of_missing.keys() 
                        if (degree_of_missing[x]>0 
                            and degree_of_missing[x]<=0.1 
                            and Judge_Feature_type(x,train_data_23)=='Class')]


#定义KNNImputer中数值填充的项目
knn_num_features_list = [x for x in degree_of_missing.keys() 
                        if (degree_of_missing[x]>0.1 
                            and Judge_Feature_type(x,train_data_23)=='Number')]

#定义KNNImputer中分类填充的项目
knn_cls_features_list = [x for x in degree_of_missing.keys() 
                        if (degree_of_missing[x]>0.1 
                            and Judge_Feature_type(x,train_data_23)=='Class')]

# 进行数据迭代
train_data_24 = train_data_23.copy()
test_data_24 = test_data_23.copy()

#执行均值填充
for feature_i in mean_impute_features_list:
    temp_mean = train_data_24.loc[:,feature_i].mean()
    train_data_24[feature_i] = train_data_24.loc[:,feature_i].fillna(temp_mean)
    test_data_24[feature_i] = test_data_24.loc[:,feature_i].fillna(temp_mean)

#执行众数填充
for feature_i in mode_impute_features_list:
    temp_mode = train_data_24.loc[:,feature_i].mode()[0]
    train_data_24[feature_i] = train_data_24.loc[:,feature_i].fillna(temp_mode)
    test_data_24[feature_i] = test_data_24.loc[:,feature_i].fillna(temp_mode)

#执行KNNImputer中数值填充的项目
KI = KNNImputer()
KI.fit(train_data_24)
data_columns_in_list = list(train_data_24.columns)
for feature_i in knn_num_features_list:
    #下面的操作是，将预测后的特征值arr按照目标feature在原Dataframe中的位置切片，然后返回给原数据集
    train_data_24[feature_i] = KI.transform(train_data_24)[:,data_columns_in_list.index(feature_i)]
    test_data_24[feature_i] = KI.transform(test_data_24)[:,data_columns_in_list.index(feature_i)]
    
#执行KNN中数值填充的项目
#先定义一个小工具，用来在列表中寻找最相似的值
def find_close_in_list(taget_value,search_list):
    min_error = max(abs(max(search_list)),abs(min(search_list)))+abs(taget_value)
    ret_value = 0
    for value_i in search_list:
        this_err = abs(taget_value-value_i)
        if this_err < min_error:
            ret_value = value_i
            min_error = this_err
    return ret_value

for feature_i in knn_cls_features_list:
    #先保存该特征下的所有属性值
    train_data_not_nan = train_data_24.loc[train_data_24.loc[:,feature_i].notnull(),:]
    test_data_not_nan = test_data_24.loc[test_data_24.loc[:,feature_i].notnull(),:]
    data_not_nan = pd.concat([train_data_not_nan,test_data_not_nan])
    feature_values = list(data_not_nan.loc[:,feature_i].drop_duplicates())
    
    #保存原始数据列
    org_col_train = train_data_24.loc[:,feature_i]
    org_col_test = test_data_24.loc[:,feature_i]
    
    #提取原始数据列的非缺失项mask
    notnan_mask_train = org_col_train.notnull()
    notnan_mask_test = org_col_test.notnull()
    
    #然后用KNN进行预测
    train_data_24[feature_i] = KI.transform(train_data_24)[:,data_columns_in_list.index(feature_i)]
    test_data_24[feature_i] = KI.transform(test_data_24)[:,data_columns_in_list.index(feature_i)]
    
    #使用原始特征值表中的值进行近似回填，因为这是分类填充，但KNN返回的是回归填充
    train_data_24[feature_i] = train_data_24.loc[:,feature_i].apply(lambda x:find_close_in_list(x,feature_values))
    test_data_24[feature_i] = test_data_24.loc[:,feature_i].apply(lambda x:find_close_in_list(x,feature_values))


In [19]:
train_data_24

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,cabin_letter,cabin_number,cabins_number_of
0,3,1,22.0,1,0,7.2500,0.336957,0.156673,0.383838,77.2,1.0
1,1,0,38.0,1,0,71.2833,0.553571,0.795276,0.593220,85.0,1.0
2,3,0,26.0,0,0,7.9250,0.336957,0.701087,0.757576,50.2,1.0
3,1,0,35.0,1,0,53.1000,0.336957,0.795276,0.593220,123.0,1.0
4,3,1,35.0,0,0,8.0500,0.336957,0.156673,0.744681,46.6,1.0
...,...,...,...,...,...,...,...,...,...,...,...
886,2,1,27.0,0,0,13.0000,0.336957,0.002569,0.383838,77.8,1.0
887,1,0,19.0,0,0,30.0000,0.336957,0.701087,0.744681,42.0,1.0
888,3,0,26.8,1,2,23.4500,0.336957,0.701087,0.744681,24.4,1.0
889,1,1,26.0,0,0,30.0000,0.553571,0.156673,0.593220,148.0,1.0


In [20]:
test_data_24

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,cabin_letter,cabin_number,cabins_number_of
0,3,1,34.5,0,0,7.8292,0.389610,0.156673,0.744681,46.6,1.0
1,3,0,47.0,1,0,7.0000,0.336957,0.795276,0.383838,79.6,1.0
2,2,1,62.0,0,0,9.6875,0.389610,0.156673,0.494491,77.4,1.0
3,3,1,27.0,0,0,8.6625,0.336957,0.156673,0.757576,50.2,1.0
4,3,0,22.0,1,1,12.2875,0.336957,0.795276,0.744681,52.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
413,3,1,30.8,0,0,8.0500,0.336957,0.156673,0.593220,51.4,1.0
414,1,0,39.0,0,0,108.9000,0.553571,0.383838,0.593220,105.0,1.0
415,3,1,38.5,0,0,7.2500,0.336957,0.156673,0.744681,46.8,1.0
416,3,1,30.8,0,0,8.0500,0.336957,0.156673,0.593220,51.4,1.0


In [21]:
#5)标准化
scaler = StandardScaler()
scaler.fit(train_data_24)
train_data_25 = pd.DataFrame(scaler.transform(train_data_24),columns=train_data_24.columns,index=train_data_24.index)
test_data_25 = pd.DataFrame(scaler.transform(test_data_24),columns=test_data_24.columns,index=test_data_24.index)


In [22]:
train_data_25

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,cabin_letter,cabin_number,cabins_number_of
0,0.827377,0.737695,-0.583050,0.432793,-0.473674,-0.502445,-0.541542,-0.811172,-1.597227,1.104129,-0.148666
1,-1.566107,-1.355574,0.616496,0.432793,-0.473674,0.786845,2.042658,1.482791,-0.163724,1.458792,-0.148666
2,0.827377,-1.355574,-0.283163,-0.474545,-0.473674,-0.488854,-0.541542,1.144451,0.961512,-0.123549,-0.148666
3,-1.566107,-1.355574,0.391581,0.432793,-0.473674,0.420730,-0.541542,1.482791,-0.163724,3.186635,-0.148666
4,0.827377,0.737695,0.391581,-0.474545,-0.473674,-0.486337,-0.541542,-0.811172,0.873229,-0.287239,-0.148666
...,...,...,...,...,...,...,...,...,...,...,...
886,-0.369365,0.737695,-0.208192,-0.474545,-0.473674,-0.386671,-0.541542,-1.364739,-1.597227,1.131411,-0.148666
887,-1.566107,-1.355574,-0.807965,-0.474545,-0.473674,-0.044381,-0.541542,1.144451,0.873229,-0.496399,-0.148666
888,0.827377,-1.355574,-0.223186,0.432793,2.008933,-0.176263,-0.541542,1.144451,0.873229,-1.296663,-0.148666
889,-1.566107,0.737695,-0.283163,-0.474545,-0.473674,-0.044381,2.042658,-0.811172,-0.163724,4.323374,-0.148666


In [23]:
test_data_25

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,cabin_letter,cabin_number,cabins_number_of
0,0.827377,0.737695,0.354096,-0.474545,-0.473674,-0.490783,0.086615,-0.811172,0.873229,-0.287239,-0.148666
1,0.827377,-1.355574,1.291241,0.432793,-0.473674,-0.507479,-0.541542,1.482791,-1.597227,1.213256,-0.148666
2,-0.369365,0.737695,2.415816,-0.474545,-0.473674,-0.453367,0.086615,-0.811172,-0.839661,1.113223,-0.148666
3,0.827377,0.737695,-0.208192,-0.474545,-0.473674,-0.474005,-0.541542,-0.811172,0.961512,-0.123549,-0.148666
4,0.827377,-1.355574,-0.583050,0.432793,0.767630,-0.401017,-0.541542,1.482791,0.873229,-0.041703,-0.148666
...,...,...,...,...,...,...,...,...,...,...,...
413,0.827377,0.737695,0.076700,-0.474545,-0.473674,-0.486337,-0.541542,-0.811172,-0.163724,-0.068985,-0.148666
414,-1.566107,-1.355574,0.691468,-0.474545,-0.473674,1.544246,2.042658,0.004842,-0.163724,2.368183,-0.148666
415,0.827377,0.737695,0.653982,-0.474545,-0.473674,-0.502445,-0.541542,-0.811172,0.873229,-0.278145,-0.148666
416,0.827377,0.737695,0.076700,-0.474545,-0.473674,-0.486337,-0.541542,-0.811172,-0.163724,-0.068985,-0.148666


In [24]:
#6)特征降维和筛选

#迭代数据
train_data_26 = train_data_25.copy()
test_data_26 = test_data_25.copy()
#特征降维
pca_demension = 11 #there are 11 features in total
pca = PCA(n_components=pca_demension)
pca.fit(train_data_26)
#train_data_26 = pca.transform(train_data_26)
print("PCA variance ratio:", pca.explained_variance_ratio_)

#特征筛选，
#filter
skb = SelectKBest(k=9)
skb.fit(train_data_26,train_target)
train_data_26 = train_data_26.loc[:,skb.get_support()]
test_data_26 = test_data_26.loc[:,skb.get_support()]
#wrapper和Embedded留给算法自己去做


PCA variance ratio: [0.24254689 0.17615668 0.12841841 0.12203715 0.08708808 0.07090749
 0.0536353  0.04611969 0.04451142 0.02391116 0.00466774]


In [25]:
train_data_26

,Pclass,Sex,Age,Parch,Fare,Embarked,Title,cabin_letter,cabins_number_of
0,0.827377,0.737695,-0.583050,-0.473674,-0.502445,-0.541542,-0.811172,-1.597227,-0.148666
1,-1.566107,-1.355574,0.616496,-0.473674,0.786845,2.042658,1.482791,-0.163724,-0.148666
2,0.827377,-1.355574,-0.283163,-0.473674,-0.488854,-0.541542,1.144451,0.961512,-0.148666
3,-1.566107,-1.355574,0.391581,-0.473674,0.420730,-0.541542,1.482791,-0.163724,-0.148666
4,0.827377,0.737695,0.391581,-0.473674,-0.486337,-0.541542,-0.811172,0.873229,-0.148666
...,...,...,...,...,...,...,...,...,...
886,-0.369365,0.737695,-0.208192,-0.473674,-0.386671,-0.541542,-1.364739,-1.597227,-0.148666
887,-1.566107,-1.355574,-0.807965,-0.473674,-0.044381,-0.541542,1.144451,0.873229,-0.148666
888,0.827377,-1.355574,-0.223186,2.008933,-0.176263,-0.541542,1.144451,0.873229,-0.148666
889,-1.566107,0.737695,-0.283163,-0.473674,-0.044381,2.042658,-0.811172,-0.163724,-0.148666


In [26]:
test_data_26

,Pclass,Sex,Age,Parch,Fare,Embarked,Title,cabin_letter,cabins_number_of
0,0.827377,0.737695,0.354096,-0.473674,-0.490783,0.086615,-0.811172,0.873229,-0.148666
1,0.827377,-1.355574,1.291241,-0.473674,-0.507479,-0.541542,1.482791,-1.597227,-0.148666
2,-0.369365,0.737695,2.415816,-0.473674,-0.453367,0.086615,-0.811172,-0.839661,-0.148666
3,0.827377,0.737695,-0.208192,-0.473674,-0.474005,-0.541542,-0.811172,0.961512,-0.148666
4,0.827377,-1.355574,-0.583050,0.767630,-0.401017,-0.541542,1.482791,0.873229,-0.148666
...,...,...,...,...,...,...,...,...,...
413,0.827377,0.737695,0.076700,-0.473674,-0.486337,-0.541542,-0.811172,-0.163724,-0.148666
414,-1.566107,-1.355574,0.691468,-0.473674,1.544246,2.042658,0.004842,-0.163724,-0.148666
415,0.827377,0.737695,0.653982,-0.473674,-0.502445,-0.541542,-0.811172,0.873229,-0.148666
416,0.827377,0.737695,0.076700,-0.473674,-0.486337,-0.541542,-0.811172,-0.163724,-0.148666


## 3.设定大致的模型参数范围 ##

In [27]:
#grid search
params = {'max_depth': -1,
          'min_data_in_leaf':20,
          'objective': 'binary',
          'num_leaves': 64,
          'learning_rate': 0.01,
          'max_bin': 512, 
          'subsample_for_bin': 200,
          'subsample': 1,
          'subsample_freq': 1,
          'colsample_bytree': 0.8,
          'reg_alpha': 5,
          'reg_lambda': 10,
          'min_split_gain': 0.5,
          'min_child_weight': 1, 
          'min_child_samples': 5,
          'scale_pos_weight': 1,
          'num_class': 1, 
          'metric': 'binary_error',
          'boosting_type':'gbdt'
         }

grid_params = {
               'max_depth': [-1]+list(range(5,20,2)),
               'min_data_in_leaf':list(range(4,40,5)),
               'n_estimators': [8, 24],
               'seed': [48],
               'learning_rate': [0.01],
               'n_estimators': [8, 24],
               'num_leaves': [6, 8, 12, 16],
               'colsample_bytree': [0.65, 0.75, 0.8],
               'subsample': [0.7, 0.75],
               'lambda_l1': [0, 1, 2, 6],
               'lambda_l2': [0, 1, 2, 6],
              }

# Create parameters to search
for i in params.keys():
    if i not in grid_params.keys():
        grid_params[i] = [params[i]]

## 4.进行交叉验证 ##

In [30]:
module = lgb.LGBMClassifier()
grid = GridSearchCV(module, param_grid=grid_params, verbose=1, cv=5, n_jobs=-1)

grid.fit(train_data_26, train_target)

# Print the best parameters found
print(grid.best_params_)
print(grid.best_score_)

#保存grid 
f=open(r'C:\Users\ZOE ZHAO\Desktop\Course\lesson8 特征工程\lightgdm_grid_module.plk','wb')  
pickle.dump(grid,f,0)  
f.close()

Fitting 5 folds for each of 55296 candidates, totalling 276480 fits
[LightGBM] [Warning] min_data_in_leaf is set=4, min_child_samples=5 will be ignored. Current value: min_data_in_leaf=4
[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=5 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=10 will be ignored. Current value: lambda_l2=0
{'boosting_type': 'gbdt', 'colsample_bytree': 0.75, 'lambda_l1': 0, 'lambda_l2': 0, 'learning_rate': 0.01, 'max_bin': 512, 'max_depth': 5, 'metric': 'binary_error', 'min_child_samples': 5, 'min_child_weight': 1, 'min_data_in_leaf': 4, 'min_split_gain': 0.5, 'n_estimators': 24, 'num_class': 1, 'num_leaves': 16, 'objective': 'binary', 'reg_alpha': 5, 'reg_lambda': 10, 'scale_pos_weight': 1, 'seed': 48, 'subsample': 0.75, 'subsample_for_bin': 200, 'subsample_freq': 1}
0.6981168790408637


In [34]:
best_params = {k: grid.best_params_.get(k, v) for k, v in params.items()}
best_params

{'boosting_type': 'gbdt',
 'max_depth': 5,
 'objective': 'binary',
 'num_leaves': 16,
 'learning_rate': 0.01,
 'max_bin': 512,
 'subsample_for_bin': 200,
 'subsample': 0.75,
 'subsample_freq': 1,
 'colsample_bytree': 0.75,
 'reg_alpha': 5,
 'reg_lambda': 10,
 'min_split_gain': 0.5,
 'min_child_weight': 1,
 'min_child_samples': 5,
 'scale_pos_weight': 1,
 'num_class': 1,
 'metric': 'binary_error'}

## 5.使用最佳参数进行训练 ##

In [39]:
#先看下跑分
x_train, x_test, y_train, y_test = train_test_split(train_data_26, train_target,test_size=0.33, random_state=42)

module = lgb.LGBMClassifier(**best_params)
module.fit(x_train,y_train)
y_pred = module.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.7898305084745763


In [42]:
#正式预测
module = lgb.LGBMClassifier(**best_params)
module.fit(train_data_26,train_target)
y_pred = module.predict(test_data_26)

sub = pd.DataFrame()
sub['PassengerId'] = test_ID
sub['Survived'] = y_pred
sub.to_csv(r'C:\Users\ZOE ZHAO\Desktop\Course\lesson8 特征工程\sub.csv', index=False)